In [ ]:
import os, sys,pytz
import django
from datetime import datetime
sys.path.append('../..') # add path to project root dir
os.environ["DJANGO_SETTINGS_MODULE"] = "flourish.settings"
from django.apps import apps as django_apps
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from django.apps import apps as django_apps

django.setup()

Remove mother from the c_sec_quart1_schedule1 schedule and put her in the c_sec_fu_quart1_schedule1 schedule,which aligns with the child ,then link the mothers schdule to the child

In [ ]:
from edc_visit_schedule.models import SubjectScheduleHistory

subject_identifier = 'B142-040990112-0'
schedule_name_previous = 'c_sec_quart1_schedule1'

subject_history_obj = SubjectScheduleHistory.objects.get(
subject_identifier=subject_identifier,
schedule_name = schedule_name_previous
)

onschedule_model_cls= django_apps.get_model(subject_history_obj.onschedule_model)

onschedule_model_obj = onschedule_model_cls.objects.get(
    subject_identifier=subject_identifier,
    schedule_name=schedule_name_previous,
    onschedule_datetime=subject_history_obj.onschedule_datetime,
)

onschedule_model_obj.delete()
subject_history_obj.delete()

In [ ]:
from edc_visit_schedule.site_visit_schedules import site_visit_schedules
from flourish_caregiver.models.onschedule import OnScheduleCohortCSecSeq
from flourish_child.models.onschedule import OnScheduleChildCohortCSecSeq

tz = pytz.timezone('Africa/Gaborone')

child_schedule = OnScheduleChildCohortCSecSeq.objects.get(subject_identifier='B142-040990112-0-10')
report_datetime = child_schedule.onschedule_datetime
report_datetime = report_datetime.astimezone(tz)
schedule_name='c_sec_fu_quart1_schedule1'

_, schedule = site_visit_schedules.get_by_onschedule_model_schedule_name(
    onschedule_model='flourish_caregiver.onschedulecohortcsecseq',name=schedule_name
)
schedule.put_on_schedule(
    subject_identifier=subject_identifier,onschedule_datetime=report_datetime,
    schedule_name=schedule_name,base_appt_datetime=report_datetime
)
print("Schedule created for",schedule)
onschdule_mother= OnScheduleCohortCSecSeq.objects.get(subject_identifier=subject_identifier)
onschdule_mother.child_subject_identifier =child_schedule.subject_identifier
onschdule_mother.save()

print("Linked with child")


